### 구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 필요한 라이브러리 임포트

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/greenDevil/진짜 쓸 거')
### Required Packages
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
import tqdm
import time
import transformers
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline
use_cuda = torch.cuda.is_available()
import transformers
from transformers import BertForSequenceClassification, BertConfig
import copy
from sklearn.metrics import average_precision_score
import medtent # 우리가 제안하는 SFDA 기법인 Med-TENT

### pickle 파일 업로드

The pickled list is a list of lists where each sublist represent a patient record that looks like
[pt_id,label, seq_list , segment_list ]


In [3]:
test_f2=pkl.load( open('/content/drive/MyDrive/greenDevil/진짜 쓸 거/labeled_data_v2/my_bertft.test_175_sepsis.pkl', 'rb'), encoding='bytes')

### 필요한 함수 정의

In [4]:
### Below are key functions for  Data prepartion ,formating input data into features, and model defintion

class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example



def convert_EHRexamples_to_features(examples,max_seq_length):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        feature = convert_singleEHR_example(ex_index, example, max_seq_length)
        features.append(feature)
    return features

### EHR version

def convert_singleEHR_example(ex_index, example, max_seq_length):
    if isinstance(example, PaddingInputExample):
        return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)

    input_ids=example[2]
    segment_ids=example[3]
    label_id=example[1]


  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
    input_mask = [1] * len(input_ids)


  # LR 5/13 Left Truncate longer sequence
    while len(input_ids) > max_seq_length:
        input_ids= input_ids[-max_seq_length:]
        input_mask= input_mask[-max_seq_length:]
        segment_ids= segment_ids[-max_seq_length:]



  # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length


    feature =[input_ids,input_mask,segment_ids,label_id,True]
    return feature

In [5]:
class BERTdataEHR(Dataset):
    def __init__(self, Features):

        self.data= Features


    def __getitem__(self, idx, seeDescription = False):

        sample = self.data[idx]

        return sample

    def __len__(self):
        return len(self.data)


#customized parts for EHRdataloader
def my_collate(batch):
        all_input_ids = []
        all_input_mask = []
        all_segment_ids = []
        all_label_ids = []

        for feature in batch:
            all_input_ids.append(feature[0])
            all_input_mask.append(feature[1])
            all_segment_ids.append(feature[2])
            all_label_ids.append(feature[3])
        return [all_input_ids, all_input_mask,all_segment_ids,all_label_ids]


class BERTdataEHRloader(DataLoader):
    def __init__(self, dataset, batch_size=128, shuffle=False, sampler=None, batch_sampler=None,
                 num_workers=0, collate_fn=my_collate, pin_memory=False, drop_last=False,
                 timeout=0, worker_init_fn=None):
        DataLoader.__init__(self, dataset, batch_size=batch_size, shuffle=False, sampler=None, batch_sampler=None,
                 num_workers=0, collate_fn=my_collate, pin_memory=False, drop_last=False,
                 timeout=0, worker_init_fn=None)
        self.collate_fn = collate_fn

##### Model Definition

In [6]:
class EHR_BERT_LR(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False ,time=False, preTrainEmb=''):
        super(EHR_BERT_LR, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        self.time=time

        if bi: self.bi=2
        else: self.bi=1

        config = BertConfig.from_json_file("/content/drive/MyDrive/greenDevil/진짜 쓸 거/hospital73_pretraining_v2/config.json")
        self.PreBERTmodel = BertForSequenceClassification(config)
        self.PreBERTmodel.load_state_dict(torch.load("/content/drive/MyDrive/greenDevil/진짜 쓸 거/converted_pytorch_model.bin", map_location='cpu'), strict=False)

        # self.PreBERTmodel=BertForSequenceClassification.from_pretrained("pretrained_py_models/45M_chk")
        if use_cuda:
           self.PreBERTmodel.cuda()
        input_size=self.PreBERTmodel.bert.config.vocab_size
        self.in_size= self.PreBERTmodel.bert.config.hidden_size

        self.dropout = nn.Dropout(p=self.dropout_r)
        self.out = nn.Linear(self.in_size,1)
        self.sigmoid = nn.Sigmoid()
        self.softmax=nn.Softmax()
        if use_cuda:
            self.flt_typ=torch.cuda.FloatTensor
            self.lnt_typ=torch.cuda.LongTensor
        else:
            self.lnt_typ=torch.LongTensor
            self.flt_typ=torch.FloatTensor

    def forward(self, sequence):
        token_t=torch.from_numpy(np.asarray(sequence[0],dtype=int)).type(self.lnt_typ)
        seg_t=torch.from_numpy(np.asarray(sequence[2],dtype=int)).type(self.lnt_typ)
        Label_t=torch.from_numpy(np.asarray(sequence[3],dtype=int)).type(self.lnt_typ)
        Bert_out=self.PreBERTmodel.bert(input_ids=token_t, attention_mask=torch.from_numpy(np.asarray(sequence[1],dtype=int)).type(self.lnt_typ),
                                    token_type_ids=seg_t)
        output=self.sigmoid(self.out(Bert_out[1]))

        return output.squeeze(),Label_t.type(self.flt_typ)

In [7]:
def timeSince(since):
   now = time.time()
   s = now - since
   m = math.floor(s / 60)
   s -= m * 60
   return '%dm %ds' % (m, s)

def trainsample(sample, model, optimizer, criterion = nn.BCELoss()):
    model.train()
    model.zero_grad()
    output,label_tensor = model(sample)
    #label_tensor = smooth_labels(label_tensor, 0.1)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
    return output, loss.item()


#train with loaders

def trainbatches(mbs_list, model, optimizer, shuffle = True):
    current_loss = 0
    all_losses =[]
    plot_every = 5
    n_iter = 0
    if shuffle:
        random.shuffle(mbs_list)
    for i,batch in enumerate(mbs_list):
        output, loss = trainsample(batch, model, optimizer, criterion = nn.BCELoss())
        current_loss += loss
        n_iter +=1

        if n_iter % plot_every == 0:
            all_losses.append(current_loss/plot_every)
            current_loss = 0
    return current_loss, all_losses


def calculate_auc(model, mbs_list, shuffle = False):
    model.eval()
    y_real =[]
    y_hat= []
    if shuffle:
        random.shuffle(mbs_list)
    for i,batch in enumerate(mbs_list):
        output,label_tensor = model(batch)
        y_hat.extend(output.cpu().data.view(-1).numpy())
        y_real.extend(label_tensor.cpu().data.view(-1).numpy())
    auc = roc_auc_score(y_real, y_hat)
    ap = average_precision_score(y_real, y_hat)

    return auc, ap, y_real, y_hat


In [8]:
def unpack_from_mycollate(batch, use_cuda=False, num_labels=1):
    """
    my_collate가 반환한 형태:
      [all_input_ids, all_input_mask, all_segment_ids, all_label_ids]
    그런데 외부에서 한 번 더 래핑되어 [[...]] 형태로 올 수 있어 그리면 한 겹 벗겨줌
    반환: (input_ids[ B,L ] Long, attention_mask[ B,L ] Long,
           token_type_ids[ B,L ] Long, labels[ B ] Float(Long))
    """
    # 한 겹 더 싸여 있으면 풀기
    if isinstance(batch, (list, tuple)) and len(batch) == 1 and isinstance(batch[0], (list, tuple)):
        batch = batch[0]

    ids_li, mask_li, seg_li, lab_li = batch  # 4개 리스트
    # 리스트/넘파이 → [B,L] LongTensor
    input_ids      = torch.as_tensor(np.asarray(ids_li),  dtype=torch.long)
    attention_mask = torch.as_tensor(np.asarray(mask_li), dtype=torch.long)
    token_type_ids = torch.as_tensor(np.asarray(seg_li),  dtype=torch.long)

    # 라벨 → [B]
    labels = torch.as_tensor(np.asarray(lab_li))
    labels = labels.float() if num_labels == 1 else labels.long()

    if use_cuda:
        input_ids = input_ids.cuda(non_blocking=True)
        attention_mask = attention_mask.cuda(non_blocking=True)
        token_type_ids = token_type_ids.cuda(non_blocking=True)
        labels = labels.cuda(non_blocking=True)

    return input_ids, attention_mask, token_type_ids, labels

###Med-TENT 적용

In [9]:
# Med-TENT version
def new_epochs_run(make_model_fn, checkpoint_path, loader, lr, use_cuda=False, num_labels=1):
    """
    make_model_fn: 동일 구조 모델 생성 함수
    checkpoint_path: fine-tuned best state_dict 경로
    loader: test2 DataLoader (my_collate 기반)
    num_labels: 1(시그모이드) 또는 2(소프트맥스)
    """
    def disable_dropout(m):
      for name, child in m.named_children():
        if isinstance(child, torch.nn.Dropout):
            setattr(m, name, torch.nn.Identity())
        else:
            disable_dropout(child)

    # 동일 가중치 모델 다시 로드 (적응 누적 방지)
    ehr = make_model_fn()
    state = torch.load(checkpoint_path, map_location=("cuda" if use_cuda else "cpu"))
    ehr.load_state_dict(state)
    if use_cuda:
        ehr.cuda()

    # Med-TENT 셋업
    base = ehr.PreBERTmodel
    disable_dropout(base)
    base = medtent.configure_model(base)                # LN만 requires_grad=True
    ln_params, ln_names = medtent.collect_params(base, top_k=4, include_classifier_bias=False)  # bias update
    # optimizer = optim.Adam(ln_params, lr=lr)
    optimizer = optim.SGD(
      ln_params,
      lr=1e-3,
      momentum=0.9,
      nesterov=True,
      weight_decay=0.0
    )

    model2 = medtent.MedTent(base, optimizer, steps=1, episodic=True)

    y_true, y_hat = [], []
    base.train()  # test-time에도 train 모드 유지

    for batch in loader:
        # my_collate → [all_ids, all_mask, all_segs, all_labels] 형태를 텐서로 정규화
        input_ids, attention_mask, token_type_ids, labels = unpack_from_mycollate(
            batch, use_cuda=use_cuda, num_labels=2
        )
        batch_dict = {"input_ids": input_ids, "attention_mask": attention_mask}

        if token_type_ids is not None:
            token_type_ids = torch.zeros_like(input_ids)

        outputs = model2(batch_dict)
        logits = outputs.logits.detach().cpu().numpy()

        if logits.ndim == 2 and logits.shape[1] == 1:
            #sigmoid
            probs = 1.0 / (1.0 + np.exp(-logits[:, 0]))
        else:
            #softmax
            e = np.exp(logits - logits.max(axis=1, keepdims=True))
            probs = (e / e.sum(axis=1, keepdims=True))[:, 1]

        y_hat.extend(probs.tolist())
        y_true.extend(labels.detach().cpu().numpy().reshape(-1).tolist())

    auc = roc_auc_score(y_true, y_hat)
    ap = average_precision_score(y_true, y_hat)
    return auc, ap, np.array(y_true), np.array(y_hat)

In [10]:
MAX_SEQ_LENGTH = 128
BATCH_SIZE = 100

test_features2 = convert_EHRexamples_to_features(test_f2, MAX_SEQ_LENGTH)
test2 = BERTdataEHR(test_features2)
test_mbs2 = list(BERTdataEHRloader(test2, batch_size = BATCH_SIZE))

In [11]:
# best 모델 state_dict 저장 경로 (epochs_run에서 저장한 경로 사용)
ckpt_path = '/content/drive/MyDrive/greenDevil/진짜 쓸 거/test_LR_Bert_BiGRU_FCfirst_runEHRmodel.st'

# test2(target domain)에 Med-TENT 적용하여 재평가
make_model = lambda: EHR_BERT_LR(input_size= 90000, embed_dim=192, hidden_size=192)
test2_auc_tent, test2_auprc_tent, y_true2, y_hat2 = new_epochs_run(
    make_model_fn=make_model,
    checkpoint_path=ckpt_path,
    loader=test_mbs2,
    lr=5e-3,
    use_cuda=use_cuda,
    num_labels=1
)

# print(f"[Baseline] Test2 AUC: {desc3[('Test_AUC2', 'mean')].iloc[0]:.4f}")
# print(f"[Baseline] Test2 AUPRC: {desc3[('Test_AUPRC2', 'mean')].iloc[0]:.4f}")
print(f"[Apply Med-TENT] Test2 AUC: {test2_auc_tent:.4f}")
print(f"[Apply Med-TENT] Test2 AUPRC: {test2_auprc_tent:.4f}")

[Apply Med-TENT] Test2 AUC: 0.6021
[Apply Med-TENT] Test2 AUPRC: 0.3603
